# 神经网络

## 人工神经元

深度学习的核心是神经网络：一种能通过简单函数的组合来表示复杂函数的数学实体。这些复杂函数的基本构件是神经元，如 `o = tanh(w * x + b)` 。其核心就是输入的线性变换（例如，将输入乘以一个数字【权重】，加上一个常数【偏置】），然后应用一个固定的非线性函数，即激活函数。从数学上讲，我们可以把这个写成 `o = f(w * x + b)` ，其中 `x` 是输入， `w` 是权重或比例因子， `b` 是偏置或偏移量， `f` 是激活函数。通常， `x` 和 `o` 可以是简单的标量或向量值（意思是保留许多标量值）。类似地， `w` 可以是单个标量或矩阵，而 `b` 是标量或向量（输入的维度和权重必须匹配）。在后一种的情况下，前面的表达式被称为神经元，因为它通过多维权重和偏置来表示许多神经元。

### 组成一个多层网络

一个多层的神经网络就由上面讨论的函数组成：

```
x_1 = f(w_0 * x + b_0)
x_2 = f(w_1 * x_1 + b_1)
...
y = f(w_n * x_n + b_n)
```
其中前一层神经元的输出被用作后一层神经元的输入。需要注意到的是 `w_0` 是一个矩阵，而 `x` 是一个向量，使用向量可以让 `w_0` 承载整个神经元层，而不是单一的权重。

### 激活函数

**激活函数** 有两个重要作用：
    
    1. 在模型内部，它允许输出函数在不同值上有不同的斜率，这是线性函数无法做到的。通过巧妙的为许多输入设置不同的斜率，神经网络可以近似任意函数。
    2. 在网络的最后一层，它的作用是将前面的线性运算输出集中到给定的范围内。
    
现在先讨论第二个作用是什么意思。假设我们正在给图像分配一个好狗狗分数，那么德牧和西班牙猎犬的图片应该得分很高，而飞机和垃圾车的得分应该很低，熊的图片得分应该也很低，虽然比垃圾车得分高。现在存在的问题是：我们必须定义一个“高分”，这意味着我们需要处理所有的 `float32` 范围，因为可能会得到相当高的分数。即使我们定义了一个十分制的量表，任然存在得分超过10分的情况，因为本质上它都是 `w * x + b` 的值，这样的函数不会自然的限定到特定的输出范围内。

最终我们希望将线性操作的输出严格的控制在一个特定的范围内，这样输出的接受者就能更好的处理接下来的操作。

对于上面问题的解决方案，一种可能是对输出值设置上限：低于 0 的值设为 0 ，高于 10 的值设为 10 。这涉及一个叫做 `torch.nn.Hardtanh()` 的激活函数（默认范围为 -1 到 1），下面是这个激活函数的图像：

![](data/images/Hardtanh.png)

另一类运行良好的函数是 `torch.nn.Sigmoid()` ，包括 `1 / (1 + e ** -x)` 、`torch.tanh()` 等。这些函数的曲线在 `x` 趋于负无穷大是逐渐接近 0 或 -1 ，在 `x==0` 时具有基本恒定的斜率。

一般来说激活函数有如下特性（在特定条件下可能是错误的）：

    * 激活函数是非线性的，非线性使得整个网络能够逼近更复杂的函数。
    * 激活函数是可微的，因此可以通过他们计算梯度。

没有这些特性，网络要么退回线性模型，要么就变得难以训练。以下是激活函数的真实情况：

    * 它们至少有一个敏感范围，在这个范围内，对输入的变化会导致输出产生相应的变化，这是训练所需要的。
    * 它们包含很多不敏感（或饱和）的范围，即输入的变化导致输出的变化很小或没有变化。

通常情况下（并非普遍如此），激活函数至少有以下一种特征：

    * 当输入到负无穷时，接近（或满足）一个下限。
    * 正无穷时相似但上界相反。

## PyTorch nn 模块

`PyTorch` 有一个专门用于神经网络的子模块，叫做 `torch.nn` ，它包含了创建各种神经网络结构所需的构建块。按照 `PyTorch` 的说法，这些构建块被称为模块（在其它框架中也被称为层）。 `PyTorch` 模块派生自基类 `nn.Module` ，一个模块可以有一个或多个参数实例作为属性，这些参数实例是张量，它们的值将在训练过程中得到优化（相当于之前线性模型中的 `w` 和 `b` ）。一个模块还可以有一个或多个子模块（`nn.Module`的子类）作为属性，并且它还能跟踪它们的参数。

现在我们可以找到一个称为 `nn.Linear` 的 `nn.module` 的子类，它通过参数属性、权重和偏置对输入应用仿射变换（仿射变换是指在几何中，一个向量空间进行一次线性变换并接上一个平移，变换为另一个向量空间 。仿射变换包括平移、旋转、放缩、剪切和反射  。它可以将一个图形在平面或空间中进行平移、旋转、缩放、错切等操作 。在仿射变换中，线性变换是指不改变原点的变换，而平移则是指将图形移动到其他位置。），它等价于之前在温度计实例中实现的。现在尝试将之前的代码转换为使用 `nn` 。

In [6]:
import torch
t_c = [0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)
suffled_indices = torch.randperm(n_samples)
train_indices = suffled_indices[:-n_val]
val_indices = suffled_indices[-n_val:]

t_u_train = t_u[train_indices]
t_c_train = t_c[train_indices]

t_u_val = t_u[val_indices]
t_c_val = t_c[val_indices]

t_un_train = 0.1 * t_u_train
t_un_val = 0.1 * t_u_val

之前的参数已经初始化完毕，现在回到线性模型，构造函数 `nn.Linear` 接受 3 个参数（实际上是 5 个，另外两个分别是设备 `device` 和类型 `dtype` ，在此不做讨论）：输入的特征数量、输出的特征数量以及线性模型是否包含偏置（默认 `True` ）。

In [8]:
import torch.nn as nn
linear_model = nn.Linear(1, 1)
linear_model

Linear(in_features=1, out_features=1, bias=True)

在这个例子中特征数量是指模块输入和输出张量的大小，因此都为 1 。例如，如果我们同时将温度和气压作为输入，我们就会有 2 个输入特征和 1 个输出特征。正如我们将要看到的对于具有多个中间模块的更复杂的模型，特征的数量将与模型的容量相关联。

目前我们只有一个 `nn.Linear` 实例，具有一个输入和一个输出特征。这只需要一个权重和一个偏置：

In [9]:
linear_model.weight, linear_model.bias

(Parameter containing:
 tensor([[-0.3428]], requires_grad=True),
 Parameter containing:
 tensor([-0.3839], requires_grad=True))

现在我们可以通过一些输入来调用模块：

In [10]:
x = torch.ones(1)
linear_model(x)

tensor([-0.7267], grad_fn=<AddBackward0>)

尽管这一次调用成功了，但实际上并没有提供正确维度的输入。